## This is the notebook of python crawler and get the data.

In [2]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import requests
import lxml
from tqdm import tqdm

In [9]:
Stock_list = ["600000", "600009", "600016", "600028", "600030", "600031",
              "600036", "600048", "600050", "600104", "600196", "600276",
              "600309", "600438", "600519", "600547", "600570", "600585",
              "600588", "600690", "600703", "600745", "600809", "600837",
              "600887", "600893", "600918", "601012", "601066", "601088",
              "601138", "601166", "601211", "601288", "601318", "601336",
              "601398", "601601", "601628", "601668", "601688", "601818",
              "601857", "601888", "601899", "601995", "603259", "603288",
              "603501", "603986"]

In [3]:
def getallpage(url):
    """
    Get the number of pages, which will make us possible to get all the news related to it.
    """
    pagedata = requests.get(url).content.decode("gbk")
    mytree = lxml.etree.HTML(pagedata)
    if pagedata.find("page_newslib"):
        data = mytree.xpath("//*[@class=\"page_newslib\"]//a[last()-1]/text()")
        return data
    else:
        return ['1']

In [4]:
def PythonCrawler(stock_id):
    stock_url = "http://stock.jrj.com.cn/share," + stock_id + ",ggxw.shtml"
    # get total page number
    total_page = int(getallpage(stock_url)[0])
    # for first page
    responce = urllib.request.urlopen(stock_url)
    content = responce.read()
    parsed = BeautifulSoup(content, "html.parser")
    # get news and time
    data = parsed.find_all('ul')[-1]
    data1 = list(data)
    NEWS = pd.DataFrame(columns = ["Time", "Content"])
    for news in tqdm(data1):
        indv_news = str(news).replace('\n', '').split('target="_blank">')[-1].replace('</i></li>', '')
        indv_news_ct = indv_news.split('</a></span><i>')
        if indv_news_ct != ['']:
            indv_news_c = indv_news_ct[0]
            indv_news_t = indv_news_ct[1].split(' ')[0]
            NEWS = NEWS.append({"Time": indv_news_t, "Content": indv_news_c}, ignore_index=True)
    # for other pages:
    if total_page != 1:
        for page in tqdm(range(2, total_page+1)):
            stock_url = "http://stock.jrj.com.cn/share," + stock_id + ",ggxw_" + str(page) +".shtml"
            responce = urllib.request.urlopen(stock_url)
            content = responce.read()
            parsed = BeautifulSoup(content, "html.parser")
            # get news and time
            data = parsed.find_all('ul')[-1]
            data1 = list(data)
            for news in tqdm(data1):
                indv_news = str(news).replace('\n', '').split('target="_blank">')[-1].replace('</i></li>', '')
                indv_news_ct = indv_news.split('</a></span><i>')
                if indv_news_ct != ['']:
                    indv_news_c = indv_news_ct[0]
                    indv_news_t = indv_news_ct[1].split(' ')[0]
                    NEWS = NEWS.append({"Time": indv_news_t, "Content": indv_news_c}, ignore_index=True)
    
    NEWS.to_csv(f"{stock_id}NLPChinese.csv", index=False, encoding="utf_8_sig")
    return NEWS

In [ ]:
for stock in tqdm(Stock_list):
    PythonCrawler(stock)

  0%|          | 0/50 [00:00<?, ?it/s]